In [ ]:
!unzip -q dataset.zip
print("Data extracted successfully!")

# Verify the paths exist before proceeding
import os
if os.path.isdir("dataset/train"):
    print("Verification successful: 'dataset/train' found.")
else:
    print("Verification failed. Check the name of your zip file and its contents.")

unzip:  cannot find or open dataset.zip, dataset.zip.zip or dataset.zip.ZIP.
Data extracted successfully!
Verification failed. Check the name of your zip file and its contents.


In [ ]:
!unzip -o -q /dataset.zip
print("Data extraction attempt complete.")

Data extraction attempt complete.


In [ ]:
!pip install split-folders

In [ ]:
import splitfolders

# Define the input and output directories
input_folder = "PlantVillage" # The folder you confirmed exists
output_folder = "split_data"   # The new folder to be created

print(f"Starting data split from '{input_folder}'...")

# Split with an 80/20 ratio for training and validation
splitfolders.ratio(
    input_folder,
    output=output_folder,
    seed=42,
    ratio=(0.8, 0.2),
    group_prefix=None
)

print(f"✅ Data split complete. New folders created at '{output_folder}'.")

Starting data split from 'PlantVillage'...


Copying files: 20639 files [00:03, 6757.90 files/s]

✅ Data split complete. New folders created at 'split_data'.


In [ ]:

# Setup and 
import os
import json
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

print("TensorFlow Version:", tf.__version__)


# 1. Data Preparation (using the corrected 'split_data' path
# 🔥 FIX: Using the folders created by the split-folders script.
train_dir = "split_data/train"
val_dir = "split_data/val"

print(f"Preparing data generators from {train_dir} and {val_dir}...")

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1.0/255)

try:
    train_data = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    val_data = val_datagen.flow_from_directory(
        val_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    num_classes = len(train_data.class_indices)
    print(f"✅ Found {num_classes} classes.")
    print("-" * 50)

except Exception as e:
    print(f"❌ FATAL ERROR: Data generators failed to initialize. Check paths and split-folders output.")
    raise e

# -----------------------------------------------------------
# 2. Model Definition (The Fixed Architecture)
# -----------------------------------------------------------
print("Defining model architecture...")
base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

model = Sequential([
    base_model,
    # 🌟 CRITICAL FIX: GlobalAveragePooling2D for stable Keras loading
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

print("Model Summary:")
model.summary()


# 3. Compile and 
print("-" * 50)
print("Compiling model...")

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint('best_model_checkpoint.keras', monitor='val_accuracy', save_best_only=True)
]

print("Starting training...")
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    callbacks=callbacks
)
print("Training finished.")


# 4. Save Model and Class 
FINAL_MODEL_NAME = "plant_disease_model_v5.keras"

model.save(FINAL_MODEL_NAME, save_format='keras')
print(f"✅ Model saved as {FINAL_MODEL_NAME}")

CLASS_INDICES_NAME = "class_indices_v5.json"
with open(CLASS_INDICES_NAME, "w") as f:
    indices = {k: int(v) for k, v in train_data.class_indices.items()}
    json.dump(indices, f)
print(f"✅ Class indices saved as {CLASS_INDICES_NAME}")


# Final Deployment 
print("-" * 50)
print("NEXT STEP: Download both new files and update your app.py:")
print(f"1. {FINAL_MODEL_NAME}")
print(f"2. {CLASS_INDICES_NAME}")
print('Then in app.py, set: MODEL_PATH = "plant_disease_model_v5.keras"')
print("-" * 50)

TensorFlow Version: 2.19.0
Preparing data generators from split_data/train and split_data/val...
Found 16504 images belonging to 15 classes.
Found 4134 images belonging to 15 classes.
✅ Found 15 classes.
--------------------------------------------------
Defining model architecture...
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Model Summary:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │         7,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,921,551 (11.14 MB)

 Trainable params: 663,567 (2.53 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

--------------------------------------------------
Compiling model...
Starting training...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 236s 424ms/step - accuracy: 0.4940 - loss: 1.6736 - val_accuracy: 0.7784 - val_loss: 0.7272
Epoch 2/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 191s 371ms/step - accuracy: 0.7830 - loss: 0.6849 - val_accuracy: 0.8384 - val_loss: 0.5107
Epoch 3/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 191s 369ms/step - accuracy: 0.8185 - loss: 0.5570 - val_accuracy: 0.8633 - val_loss: 0.4261
Epoch 4/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 190s 369ms/step - accuracy: 0.8465 - loss: 0.4790 - val_accuracy: 0.8728 - val_loss: 0.3896
Epoch 5/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 195s 377ms/step - accuracy: 0.8593 - loss: 0.4254 - val_accuracy: 0.8817 - val_loss: 0.3657
Epoch 6/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 191s 369ms/step - accuracy: 0.8727 - loss: 0.3767 - val_accuracy: 0.8880 - val_loss: 0.3376
Epoch 7/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 196s 379ms/step - accuracy: 0.8797 - loss: 0.3706 - val_accuracy: 0.9008 - val_loss: 0.3061
Epoch 8/10
516/516 ━━━━━━━━━━━━━━━━━━━━ 191s 370ms/step - accuracy: 0.8884 -

Training finished.
✅ Model saved as plant_disease_model_v5.keras
✅ Class indices saved as class_indices_v5.json
--------------------------------------------------
NEXT STEP: Download both new files and update your app.py:
1. plant_disease_model_v5.keras
2. class_indices_v5.json
Then in app.py, set: MODEL_PATH = "plant_disease_model_v5.keras"
--------------------------------------------------


In [ ]:
model.save('plant_disease_model_v5.h5')

In [ ]:
from google.colab import files

files.download('plant_disease_model_v5.keras')
files.download('class_indices_v5.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>